In [2]:
from riscv_assembler.convert import AssemblyConverter

In [45]:
import os

TMP_FNAME = "tmp.s"
DST_TMP_DIR = "tmp/txt"
DST_TMP_FNAME = "tmp/txt/tmp.txt"

with open(TMP_FNAME, "w") as f:
    f.write(INSTRS)
try:
    os.remove(DST_TMP_FNAME)
    os.rmdir(DST_TMP_DIR)
except:
    pass
AssemblyConverter(output_type="t").convert(TMP_FNAME)

print()
print()
print()
print()
print()

asm_lines = [x.strip() for x in ASM.split("\n")][1:-1]
instr_lines = None

with open(DST_TMP_FNAME) as f:
    instr_lines = [x.strip() for x in f.readlines()]

iasm = 0
iinstr = 0
while iasm < len(asm_lines):
    asm = asm_lines[iasm]
    iasm += 1
    if asm.endswith(":"):
        print(f"// {asm}")
        continue
        
    instr = instr_lines[iinstr]
    iinstr += 1
    print(f"`i(32'b{instr}); // {asm}")



0


KeyError: '1'

In [3]:
import bitstring

In [43]:
def fp32_as_u32(value):
    return int(bitstring.BitArray(float=4.0, length=32).bin, 2)

def set_reg_raw(reg_name, value):
    global INSTRS
    INSTRS += f"xor {reg_name}, {reg_name}, {reg_name}\n"
    upper = (value + 0x7ff) >> 12
    lower = value - (upper << 12)
    INSTRS += f"lui {reg_name}, {upper}\n"
    INSTRS += f"addi {reg_name}, {reg_name}, {lower}\n"

def set_reg_fp32(reg_name, value):
    set_reg_raw(reg_name, fp32_as_u32(value))

def store_reg(reg_name, pos_in_word):
    global INSTRS
    INSTRS += f"sw {reg_name}, {pos_in_word * 4}(zero)\n"

def label(lb):
    global INSTRS
    INSTRS += f"{lb}:\n"

def goto(lb):
    global INSTRS
    INSTRS += f"j {lb}\n"


In [44]:
INSTRS = ""
label("begin")
set_reg_raw("t0", 0xfff)
store_reg("t0", 2)
goto("begin")
print(INSTRS)

begin:
xor t0, t0, t0
lui t0, 1
addi t0, t0, -1
sw t0, 8(zero)
j begin



In [55]:
import bitstring

def int2bits(x, length):
    return str(bitstring.BitArray(int=(int(x)), length=length))[2:]

REG_DICT = {
    "zero": "x0",
    "ra": "x1",
    "sp": "x2",
    "gp": "x3",
    "tp": "x4",
    "t0": "x5",
    "t1": "x6",
    "t2": "x7",
    "fp": "x8",
    "s0": "x8",
    "s1": "x9",
    "a0": "x10",
    "a1": "x11",
    "a2": "x12",
    "a3": "x13",
    "a4": "x14",
    "a5": "x15",
    "a6": "x16",
    "a7": "x17",
    "s2": "x18",
    "s3": "x19",
    "s4": "x20",
    "s5": "x21",
    "s6": "x22",
    "s7": "x23",
    "s8": "x24",
    "s9": "x25",
    "s10": "x26",
    "s11": "x27",
    "t3": "x28",
    "s4": "x29",
    "s5": "x30",
    "s6": "x31",
}

class Bits:
    def __init__(self, x, length):
        if isinstance(x, Bits):
            assert len(x.bits) == length
            self.bits = x.bits
        elif isinstance(x, list):
            assert all(isinstance(a, Bits) for a in x)
            assert sum(len(a.bits) for a in x) == length
            self.bits = ''.join(Bits(a).bits for a in reversed(x))
        elif isinstance(x, str):
            assert all(c == '0' or c == '1' for c in x)
            assert len(x) == length
            self.bits = x
        elif isinstance(x, int):
            self.bits = int2bits(x, length)
        else:
            raise AssertionError("unsupported bitfield")

    def __repr__(self):
        return self.bits

class NearLabel(Bits):
    def __init__(self, cur, lb):
        super(Bits, self).__init__(LABELS[lb] - (cur + 1) * 4, 13)
        
class FarLabel(Bits):
    def __init__(self, cur, lb):
        super(Bits, self).__init__(LABELS[lb] - (cur + 1) * 4, 21)

class Opcode(Bits):
    def __init__(self, opcode):
        super(Bits, self).__init__([Bits('11', 2), Bits(opcode, 5)], 7)

class Reg(Bits):
    def __init__(self, name):
        if reg_name in REG_DICT:
            reg_name = REG_DICT[reg_name]
        assert reg_name[0] == 'x'
        self.name = name
        bits = int2bits(reg_name[1:], length=5)
        super(Bits, self).__init__(bits, 5)

INSTRS = []
def make_instr(*components):
    global INSTRS
    INSTRS += [Bits(*components, 32)]

def make_rv32i_instr(opcode, *components):
    make_instr(Opcode(opcode), *components)

def make_rv32i_r_instr(opcode, rd, funct3, rs1, rs2, funct7):
    make_rv32i_instr(opcode,
                     Reg(rd),
                     Bits(funct3, 3),
                     Reg(rs1),
                     Reg(rs2),
                     Bits(funct, 7))

def make_rv32i_i_instr(opcode, rd, funct3, rs1, imm12):
    make_rv32i_instr(opcode,
                     Reg(rd),
                     Bits(funct3, 3),
                     Reg(rs1),
                     Bits(imm12, 12))

def make_rv32i_s_instr(opcode, funct3, rs1, rs2, imm12):
    imm12 = Bits(imm12, 12)
    make_rv32i_instr(opcode,
                     Bits(imm12.bits[0:5], 5),
                     Bits(funct3, 3),
                     Reg(rs1),
                     Reg(rs2),
                     Bits(imm12.bits[5:12], 7))

def make_rv32i_b_instr(opcode, funct3, rs1, rs2, imm13):
    assert imm13[0] == '0', "immediate of b-type instruction must align to 2 bytes"
    imm13 = Bits(imm13, 12)
    make_rv32i_instr(opcode,
                     Bits(imm13.bits[11], 1),
                     Bits(imm13.bits[1:5], 4),
                     Bits(funct3, 3),
                     Reg(rs1),
                     Reg(rs2),
                     Bits(imm13.bits[5:11], 6),
                     Bits(imm13.bits[12], 1))

def make_rv32i_u_instr(opcode, rd, funct3, rs1, imm20):
    make_rv32i_instr(opcode,
                     Reg(rd),
                     Bits(imm20, 20))

def make_rv32i_j_instr(opcode, rd, funct3, rs1, imm21):
    assert imm21[0] == '0', "immediate of j-type instruction must align to 2 bytes"
    make_rv32i_instr(opcode,
                     Reg(rd),
                     Bits(imm20.bits[12:20], 8),
                     Bits(imm20.bits[11], 1),
                     Bits(imm20.bits[1:11], 10),
                     Bits(imm20.bits[20], 1))


def op(f):
    def inner(*args):
        def inner_inner():
            return f(*args)
        return inner_inner
    return inner

LABELS = []
def declare_label(lb):
    global LABELS, INSTRS
    LABELS[lb] = len(INSTRS) * 4



def make_rv32i_branch_instr(funct3, rs1, rs2, imm13):
    make_rv32i_b_instr(0x18, funct3, rs1, rs2, imm13)

def beq(rs1, rs2, lb):
    make_rv32i_branch_instr(0, rs1, rs2, imm13)


def make_rv32i_int_reg_imm_instr(rd, funct3, rs1, imm12):
    make_rv32i_i_instr(0x04, funct3, rs1, imm12)
def make_rv32i_int_reg_imm_logic_shift_instr(rd, funct3, rs1, shamt):
    imm12 = [Bits(0, 6), Bits(shamt, 6)]
    make_rv32i_int_reg_imm_instr(rd, funct3, rs1, imm12)
def make_rv32i_int_reg_imm_arith_shift_instr(rd, funct3, rs1, shamt):
    imm12 = [Bits(16, 6), Bits(shamt, 6)]
    make_rv32i_int_reg_imm_instr(rd, funct3, rs1, imm12)

def addi(rd, rs1, imm12):
    make_rv32i_int_reg_imm_instr(rd, 0, rs1, imm12)
def slli(rd, rs1, shamt):
    make_rv32i_int_reg_imm_logic_shift_instr(rd, 1, rs1, shamt)
def slti(rd, rs1, imm12):
    make_rv32i_int_reg_imm_instr(rd, 2, rs1, imm12)
def sltiu(rd, rs1, imm12):
    make_rv32i_int_reg_imm_instr(rd, 3, rs1, imm12)
def xori(rd, rs1, imm12):
    make_rv32i_int_reg_imm_instr(rd, 4, rs1, imm12)
def srli(rd, rs1, shamt):
    make_rv32i_int_reg_imm_logic_shift_instr(rd, 5, rs1, shamt)
def srla(rd, rs1, shamt):
    make_rv32i_int_reg_imm_arith_shift_instr(rd, 5, rs1, shamt)
def ori(rd, rs1, imm12):
    make_rv32i_int_reg_imm_instr(rd, 6, rs1, imm12)
def andi(rd, rs1, imm12):
    make_rv32i_int_reg_imm_instr(rd, 7, rs1, imm12)


def make_rv32i_int_reg_reg_instr(rd, funct3, rs1, rs2):
    make_rv32i_r_instr(0x0C, rd, funct3, rs1, rs2, 0)
def make_rv32i_int_reg_reg_alt_instr(rd, funct3, rs1, rs2):
    make_rv32i_r_instr(0x0C, rd, funct3, rs1, rs2, 32)

def add(rd, rs1, rs2):
    make_rv32i_int_reg_reg_instr(rd, 0, rs1, rs2)
def sub(rd, rs1, rs2):
    make_rv32i_int_reg_reg_alt_instr(rd, 0, rs1, rs2)
def sll(rd, rs1, rs2):
    make_rv32i_int_reg_reg_instr(rd, 1, rs1, rs2)
def slt(rd, rs1, rs2):
    make_rv32i_int_reg_reg_instr(rd, 2, rs1, rs2)
def sltu(rd, rs1, rs2):
    make_rv32i_int_reg_reg_instr(rd, 3, rs1, rs2)
def xor(rd, rs1, rs2):
    make_rv32i_int_reg_reg_instr(rd, 4, rs1, rs2)
def srl(rd, rs1, rs2):
    make_rv32i_int_reg_reg_instr(rd, 5, rs1, rs2)
def sra(rd, rs1, rs2):
    make_rv32i_int_reg_reg_alt_instr(rd, 5, rs1, rs2)
def or_(rd, rs1, rs2):
    make_rv32i_int_reg_reg_instr(rd, 6, rs1, rs2)
def and_(rd, rs1, rs2):
    make_rv32i_int_reg_reg_instr(rd, 7, rs1, rs2)






# When instruction programming is finished, apply this.
def finish():
    return [instr() for instr in INSTRS]